I wanted to keep the lecture notebook and seperate out the wine stuff here since it upset me that we were
re-setting variable names each time we changed data sets. And also to avoid running the newsgroups cells
since that took a long time. 

In [21]:
# Import Statements
from sklearn.pipeline import Pipeline 
from sklearn.datasets import fetch_20newsgroups # demo data set 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd 


In [13]:
# load the data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train.head(1)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1


# Brute Force Run on TfidVectorizer

In [34]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
clf = RandomForestClassifier()
pipe = Pipeline([('vect', vect), ('clf', clf)])
target = 'ratingCategory'
features = 'description'
X_train = train[features]
y_train = train[target]

# model parameters
rfc_params = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (0.02, 0.05),
    'vect__max_features': (500,1000),
    'clf__n_estimators': (5,10),
    'clf__max_depth':(5,10,15,20)
}

In [35]:
# grid search
grid_search = GridSearchCV(pipe, rfc_params, cv=5, n_jobs=8, verbose=1)
grid_search.fit(X_train, y_train)
grid_search.best_score_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   13.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   58.5s
[Parallel(n_jobs=8)]: Done 320 out of 320 | elapsed:  1.7min finished


0.7173966234401762

In [57]:
# random search, working noticeably faster
random_search = RandomizedSearchCV(pipe, rfc_params, cv=5, n_jobs=8, verbose=1)
random_search.fit(X_train, y_train)
random_search.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   11.9s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:   15.8s finished


0.7107903107413751

# SVD 

In [ ]:
# instantiate SVD, removed algorithm arg since it defaults to whatever is most efficient
svd = TruncatedSVD(n_components=100, n_iter=5)

svd_params = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

# create two seperate pipelines
lsi_pipe = Pipeline([('vect', vect), ('svd', svd)])
svd_pipe = Pipeline([('lsi', lsi), ('clf', clf)])

# print(svd_pipe)

# SVD GRID SEARCH

In [44]:
svd_grid_search = GridSearchCV(svd_pipe, svd_params, cv=5, n_jobs=8, verbose=1)
svd_grid_search.fit(X_train, y_train)
svd_grid_search.best_score_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done 135 out of 135 | elapsed: 11.5min finished


0.7213114754098361

# SVD RANDOM SEARCH

In [51]:
svd_random_search = RandomizedSearchCV(svd_pipe, svd_params, cv=5, n_jobs=4, verbose=1)
svd_random_search.fit(X_train, y_train)
svd_random_search.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  3.4min finished


0.7169072669439687

# Function to create Submissions

In [62]:
def create_submission(search, subNumber):
    pred = search.predict(test['description'])
    submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
    submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
    assert(submission.shape == (1022, 2))
    submission.to_csv(f'./data/submission{str(subNumber)}.csv', index=False)

In [63]:
create_submission(grid_search, 1)

In [65]:
create_submission(random_search, 2)

In [66]:
create_submission(svd_grid_search, 3)

In [64]:
create_submission(svd_random_search, 4)